# 参数管理  
我们首先看一下具有单隐藏层的多层感知机。  

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3542],
        [-0.4339]], grad_fn=<AddmmBackward0>)

### 1. 参数访问  
当通过Sequential类定义模型时，我们可以通过索引来访问模型的任意层。  

In [2]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.1089, -0.0038,  0.3603,  0.1694],
        [-0.1750, -0.4148, -0.4396, -0.2775],
        [-0.2331,  0.2036, -0.3309, -0.2425],
        [-0.1785,  0.2878, -0.2789,  0.2824],
        [ 0.4131, -0.1963,  0.0323,  0.0753],
        [-0.2115,  0.0637, -0.3858, -0.1955],
        [-0.0635, -0.0113, -0.1989, -0.2926],
        [ 0.1229, -0.4404,  0.4611,  0.4819]])), ('bias', tensor([ 0.1816,  0.4186,  0.1141, -0.2985, -0.4952,  0.4142, -0.2229, -0.0704]))])


每个参数都表示为参数类的一个实例，可以先提取参数，然后提取参数值。  

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2875], requires_grad=True)
tensor([-0.2875])


除了值之外，我们还可以访问每个参数的梯度。

In [4]:
net[2].weight.grad == None

True

一次性访问所有参数。下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。  
**注意：单独提取出参数后，其形状与模型参数矩阵$\bold W$是转置关系**。

In [5]:
print([(name, param.shape) for name, param in net[0].named_parameters()])
print([(name, param.shape) for name, param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


因此，可以由参数名获取参数值。

In [6]:
print(net.state_dict())
print(net.state_dict()['2.weight'])
print(net.state_dict()['2.weight'].data)

OrderedDict([('0.weight', tensor([[ 0.1089, -0.0038,  0.3603,  0.1694],
        [-0.1750, -0.4148, -0.4396, -0.2775],
        [-0.2331,  0.2036, -0.3309, -0.2425],
        [-0.1785,  0.2878, -0.2789,  0.2824],
        [ 0.4131, -0.1963,  0.0323,  0.0753],
        [-0.2115,  0.0637, -0.3858, -0.1955],
        [-0.0635, -0.0113, -0.1989, -0.2926],
        [ 0.1229, -0.4404,  0.4611,  0.4819]])), ('0.bias', tensor([ 0.1816,  0.4186,  0.1141, -0.2985, -0.4952,  0.4142, -0.2229, -0.0704])), ('2.weight', tensor([[-0.1081, -0.2239, -0.0327,  0.1938, -0.1814, -0.1915,  0.1266, -0.2580]])), ('2.bias', tensor([-0.2875]))])
tensor([[-0.1081, -0.2239, -0.0327,  0.1938, -0.1814, -0.1915,  0.1266, -0.2580]])
tensor([[-0.1081, -0.2239, -0.0327,  0.1938, -0.1814, -0.1915,  0.1266, -0.2580]])


### 2. 嵌套块参数访问  

如果我们将多个块相互嵌套，参数命名约定是如何工作的。我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。  

**每个块都有自己的名字和值**。  
**每个参数都有自己的名字和值**。  

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 嵌套
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1)) # 再次嵌套
rgnet(X)
print(rgnet) # 共有三层Sequential

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。  

In [8]:
rgnet[0][1][0].bias.data

tensor([ 0.0382, -0.1143,  0.4346, -0.0233,  0.3783,  0.1351, -0.2008, -0.4892])

### 3. 参数初始化  
默认情况下， PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算出的。 PyTorch的nn.init模块提供了多种预置初始化方法。

a.内置初始化  

In [9]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0019, -0.0033, -0.0010,  0.0091]), tensor(0.))

In [10]:
# 还可以将所有参数初始化为给定的常数，比如初始化为1。
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [11]:
# 我们还可以对某些块应用不同的初始化方法。
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3664,  0.5666,  0.4568, -0.1066])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


b. 自定义初始化  
有时，深度学习框架没有提供我们需要的初始化方法。在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：  
$$
w\sim
\begin{cases}
U(5, 10)&\text{可能性}\frac 14 \\
0&\text{可能性}\frac 12 \\
U(-10, -5)&\text{可能性}\frac 14
\end{cases}
$$

In [12]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", [(name, param.shape) 
                       for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 将(-5, 5)之间的值改为0
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight.data[:3]

Init ('weight', torch.Size([8, 4]))
Init ('weight', torch.Size([1, 8]))


tensor([[-0.0000, -0.0000, -0.0000,  8.9927],
        [ 9.1204, -0.0000,  0.0000, -8.8238],
        [-7.5422, -0.0000,  7.3647,  7.4980]])

注意，我们始终可以直接设置参数。

In [13]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[:3]

tensor([[42.0000,  1.0000,  1.0000,  9.9927],
        [10.1204,  1.0000,  1.0000, -7.8238],
        [-6.5422,  1.0000,  8.3647,  8.4980]])

### 4. 参数共享（绑定）  
有时我们希望在多个层间共享参数：我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [14]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))

net(X)

# 检查参会是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])

# 参数共享：同一个对象，而不只是有相同的值
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这里有一个问题：当参数绑定时，梯度会发生什么情况？  
由于模型参数包含梯度，因此在反向传播期间第二个隐藏层（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。  

练习：构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。

### 5. 延迟初始化  
延后初始化（defers initialization），即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。  

延迟初始化可以做到：
- 定义网络时，不指定输入维度  
- 添加层时，不需要知道上一层的输出维度

初始化一个MLP

In [44]:
net = nn.Sequential(nn.LazyLinear(32), nn.ReLU(), nn.LazyLinear(10))


此时，因为输入维数是未知的，所以网络不可能知道输入层权重的维数。因此，框架尚未初始化任何参数，我们通过尝试访问以下参数进行确认。  

In [45]:
net[0].weight

<UninitializedParameter>

因此，必须让数据通过一次网络，才能知道网络的结构，并进行参数初始化。

In [46]:
X = torch.rand(2, 20)
net(X)

print(net[0].weight.data[0])
print(net[0].weight.shape)

tensor([-0.1414, -0.0298,  0.1279, -0.1899, -0.0156,  0.0108,  0.0607, -0.1772,
        -0.1525,  0.1257, -0.1940, -0.0587,  0.0096,  0.0712, -0.1048,  0.1681,
         0.1338,  0.0594, -0.0105,  0.2096])
torch.Size([32, 20])
